# Building Your First Campaign

This notebook will walk you through the steps to build a recommendation model for Allstore based on data collected from Amazon Outdoor equipment dataset. The goal is to recommend Amazon outdoor products that are relevant based on a particular user.

Note: You can bring you own dataset, but you need to ensure the schema of your dataset matches the intructions in this notebook.

## How to Use the Notebook

Code is broken up into cells like the one below. There's a triangular `Run` button at the top of this page you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.


Simply follow the instructions below and execute the cells to get started with Amazon Personalize.

## Imports 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like boto3(The AWS SDK) and Pandas/Numpy which are core data science tools

In [ ]:
# Python Built-Ins:
import json
import os
import time

# External Dependencies:
import boto3
import numpy as np
import pandas as pd


## Connect to the Personalize SDK
Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.


In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

#### Create Personalize Role

Also Amazon Personalize needs the ability to assume Roles in AWS in order to have the permissions to execute certain tasks, the lines below grant that.

In [ ]:
sts = boto3.client('sts')
response = sts.get_caller_identity()
accountid = response['Account']

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    )

    # AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
    # if you would like to use a bucket with a different name, please consider creating and attaching a new policy
    # that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
    policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = policy_arn
    )

    # Now add S3 support
    iam.attach_role_policy(
        PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
        RoleName=role_name
    )
    time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

    role_arn = create_role_response["Role"]["Arn"]
    print(role_arn)
except:
    role_arn = 'arn:aws:iam::'+accountid+':role/PersonalizeRoleDemo'
    print(role_arn)

## Configure the Data

Data is imported into Amazon Personalize through Amazon S3. We'll use the "staging" bucket that CloudFormation created for us, for both data preparation and final upload.

CloudFormation has automatically:

* Created a bucket for us to work in and a set of Lambda functions implementing the web API.
* Written these IDs to this notebook's [lifecycle configuration script](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-lifecycle-config.html), to expose them to us here as environment variables.

...So we can just load them from environment variables.


In [ ]:
bucket = os.environ["STAGING_BUCKET"]
print(f"Bucket name: {bucket}")

Rerank_arn = os.environ["LAMBDA_RERANK_ARN"]
GetRecommendationsByItem_arn = os.environ["LAMBDA_GETITEMRECS_ARN"]
GetRecommendations_arn = os.environ["LAMBDA_GETRECS_ARN"]
PostClickEvent_arn = os.environ["LAMBDA_POSTCLICK_ARN"]


In [ ]:
interactions_filename = "useritems-interactions.csv"
items_filename = "items_w_metadata.csv"


### Upload Training Data

At present you do not have the Allstore data loaded locally yet for examination, execute the lines below to upload the Dataset to your staging S3 bucket

In [ ]:
!echo "uploading Dataset to Amazon s3"
!aws s3 cp ./Datasets s3://$bucket/ --recursive --exclude ".*"

### Explore the Dataset

Users-Items Interaction dataset

In [ ]:
ratinguri = f's3://{bucket}/useritems-interactions.csv'
useritems = pd.read_csv(ratinguri, names=['USER_ID', 'ITEM_ID', 'TIMESTAMP'], header=1)
pd.set_option('display.max_rows', 4) #change the bucket name to your bucket name

useritems

Items and metadata dataset

In [ ]:
ratinguri = f's3://{bucket}/items_w_metadata.csv'
items = pd.read_csv(ratinguri, names=['asin', 'title', 'img_url','genre'], header=1)
pd.set_option('display.max_rows', 4) #change the bucket name to your bucket name

items

user dataset

In [ ]:
ratinguri = f's3://{bucket}/users.csv'
users = pd.read_csv(ratinguri, names=['userid'], header=1)
pd.set_option('display.max_rows', 4) #change the bucket name to your bucket name

users

### Create Schema

A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

try:
    create_schema_response = personalize.create_schema(
        name = "personalize-demo-usersitems-interactions",
        schema = json.dumps(interactions_schema)
    )

    interactions_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))
except:
    interactions_schema_arn = 'arn:aws:personalize:us-east-1:' + accountid + ':schema/personalize-demo-usersitems-interactions'

In [ ]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TITLE",
            "type": "string"
        },
        {
            "name": "IMGURL",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}



try:
    create_schema_response = personalize.create_schema(
        name = "personalize-demo-items",
        schema = json.dumps(items_schema)
    )

    items_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))
except:
    items_schema_arn = 'arn:aws:personalize:us-east-1:' + accountid + ':schema/personalize-demo-items'
    
items_schema_arn

### Create and Wait for Dataset Group

The largest grouping in Personalize is a Dataset Group, this will isolate your data, event trackers, solutions, and campaigns. Grouping things together that share a common collection of data. Feel free to alter the name below if you'd like.

#### Create Dataset Group

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-launch-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Wait for Dataset Group to Have ACTIVE Status

Before we can use the Dataset Group in any items below it must be active, execute the cell below and wait for it to show active.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Create Dataset User-Item Interactions Dataset

After the group, the next thing to create is the actual datasets, in this example we will only create 1 for the interactions data. Execute the cells below to create it.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-launch-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-launch-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### Attach Policy to S3 Bucket

Amazon Personalize needs to be able to read the content of your S3 bucket that you created earlier. The lines below will do that.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

## Import the data

Earlier you created the DatasetGroup and Dataset to house your information, now you will execute an import job that will load the data from S3 into Amazon Personalize for usage building your model.

#### Create Dataset Import Job Interactions

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-interactions",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, interactions_filename)
    },
    roleArn = role_arn
)

interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### Wait for Dataset Import Job to Have ACTIVE Status

It can take a while before the import job completes, please wait until you see that it is active below.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Create Dataset Import Job Items

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-items",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, items_filename)
    },
    roleArn = role_arn
)

items_interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### Wait for Dataset Import Job to Have ACTIVE Status

It can take a while before the import job completes, please wait until you see that it is active below.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Create the Solution and Version

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.

### Select Recipe

In [ ]:
list_recipes_response = personalize.list_recipes()
hrnn_arn = list_recipes_response['recipes'][0]['recipeArn']
rerank_arn = list_recipes_response['recipes'][3]['recipeArn']
sims_arn = list_recipes_response['recipes'][5]['recipeArn']
list_recipes_response

Here you can see below that we are picking the `HRNN` recipie.

# HRNN Solution

### Create and Wait for Solution

First you will create the solution with the API, then you will create a version. It will take several minutes to train the model and thus create your version of a solution. Once it gets started and you are seeing the in progress notifications it is a good time to take a break, grab a coffee, etc.

#### Create Solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = hrnn_arn
)

hrnn_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Create Solution Version

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

hrnn_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### Wait for Solution Version to Have ACTIVE Status

This will take at least 20 minutes.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more compelx datasets you should see improvements.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is simply a hosted copy of your model. Again there will be a short wait so after executing you can take a quick break while the infrastructure is being provisioned.

#### Create Campaign

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-camp-hrnn",
    solutionVersionArn = hrnn_solution_version_arn,
    minProvisionedTPS = 1
)

hrnn_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Wait for Campaign to Have ACTIVE Status

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = hrnn_campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Get Sample Recommendations

After the campaign is active you are ready to get recommendations. First we need to select a random user from the collection. Then we will create a few helper functions for getting the Allstore merchandise (item descriptions) information to show for recommendations instead of just IDs.

In [ ]:
# Getting a random user:
user_id, item_id, _ = useritems.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
# First load items into memory
allitemuri = f's3://{bucket}/items_w_metadata.csv'
items = pd.read_csv(allitemuri, sep=',', usecols=[0,1], names=['asin', 'title'], index_col='asin',header=0)

# print(items)

def get_allstore_products(asin):
    """
    Takes in an ID, returns a title
    """
    asin = str(asin)
    return items.loc[asin]['title']

#### Call GetRecommendations

Using the user that you obtained above, the lines below will get recommendations for you and return the list of movies that are recommended.


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    try :
        title = get_allstore_products(item['itemId'])
    except:
        continue
    recommendation_list.append(title)  
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Item Description'])
recommendations_df

## Review

Using the codes above you have successfully trained a deep learning model to generate Allstore Merchandise recommendations based on prior user behavior. Think about other types of problems where this data is available and what it might look like to build a system like this to offer those recommendations.

Now you are ready to move onto the next notebook `2.Building_Campaign_SIMS.ipynb`



## Update Lambda Configuration

In [ ]:
lambdaclient = boto3.client('lambda')

response = lambdaclient.get_function(
    FunctionName=GetRecommendations_arn,
)

Environment = response['Configuration']['Environment']
print(Environment)

#update Env var

Environment['Variables']['Campaign_ARN'] = hrnn_campaign_arn

response = lambdaclient.update_function_configuration(
    FunctionName=GetRecommendations_arn,
    Environment=Environment
)

## Notes for the Next Notebook:

There are a few values you will need for the next notebook, execute the cells below to store them so they can be copied and pasted into the next part of the exercise.

In [ ]:
%store Rerank_arn
%store GetRecommendationsByItem_arn
%store PostClickEvent_arn
%store dataset_group_arn
%store hrnn_campaign_arn
%store interactions_dataset_arn
%store items_dataset_arn
%store useritems
%store recommendations_df
%store bucket
%store user_id
